### Using *scaled* data
First I am importing necessary libraries, then importing data, removing the arbitrary index "Unnamed: 0" column, and viewing the first few observations for each.

In [60]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler

In [48]:
train = pd.read_csv('data/red_wine_train.csv')
train.drop('Unnamed: 0',axis=1, inplace=True)
train.head()

,fixed acidity,volatile acidity,citric acid,chlorides,total sulfur dioxide,density,sulphates,alcohol,quality,alcohol_higher,va_high
0,7.0,0.685,0.00,0.067,63.0,0.99790,0.81,9.90,5,0,1
1,8.6,0.685,0.10,0.092,12.0,0.99745,0.65,9.55,6,0,1
2,5.6,0.660,0.00,0.087,11.0,0.99378,0.63,12.80,7,1,1
3,7.7,0.510,0.28,0.087,54.0,0.99800,0.74,9.20,5,0,0
4,8.7,0.310,0.46,0.059,25.0,0.99660,0.76,10.10,6,0,0


In [49]:
test = pd.read_csv('data/red_wine_test.csv')
test.drop('Unnamed: 0',axis=1, inplace=True)
test.head()

,fixed acidity,volatile acidity,citric acid,chlorides,total sulfur dioxide,density,sulphates,alcohol,quality,alcohol_higher,va_high
0,12.6,0.310,0.72,0.072,29.0,0.99870,0.82,9.8,8,0,0
1,11.8,0.330,0.49,0.093,80.0,1.00020,0.76,10.7,7,0,0
2,7.1,0.875,0.05,0.082,14.0,0.99808,0.52,10.2,3,0,1
3,9.0,0.800,0.12,0.083,28.0,0.99836,0.65,10.4,6,0,1
4,7.9,0.690,0.21,0.080,141.0,0.99620,0.51,9.9,5,0,1


To decide how to scale each attribute, I am viewing the statistical summary of each column with describe(). Some columns already range from 0 to 1 while others have maximums above 10 or for total sulfur dioxide, almost 300. To ensure the neural network performs ideally, I am going to scale each column to [0,1] using the Min-Max Normalization method through scikitlearn's MinMaxScaler. Since we already split the data into test and train sets, I am first going to recombine these two disjoint dataframes back into one dataframe so that my values for mins and maxes needed for the formula are constant across scaling each of the two dataframes--it is likely that test and/or train contains a unique min or max value, and if we performed the min and max fit on these dfs separately, the values would not be the same (if one had a lower max for example) and our scaling would be flawed. This way, two equal values will certainly be transformed to the same value whether or not they begin in test or in train.

As mentioned, first we concatenate test and train back into one dataframe. Then we use .describe() to view summary of each column.

In [63]:
#recombining train & test to get overall max and min values so test and train are scaled w same values in the scaler
whole_set = pd.concat([train,test])

train.describe() #summary to understand all columns have varying scales

,fixed acidity,volatile acidity,citric acid,chlorides,total sulfur dioxide,density,sulphates,alcohol,quality,alcohol_higher,va_high
count,1279.000000,1279.000000,1279.000000,1279.000000,1279.000000,1279.000000,1279.000000,1279.000000,1279.000000,1279.000000,1279.000000
mean,8.270837,0.530328,0.263628,0.087603,46.213839,0.996718,0.655395,10.418647,5.637217,0.151681,0.351056
std,1.730139,0.176907,0.194559,0.047631,32.942827,0.001908,0.171623,1.069182,0.808633,0.358852,0.477487
min,4.600000,0.120000,0.000000,0.012000,6.000000,0.990070,0.330000,8.400000,3.000000,0.000000,0.000000
25%,7.100000,0.400000,0.090000,0.070000,22.000000,0.995590,0.550000,9.500000,5.000000,0.000000,0.000000
50%,7.900000,0.530000,0.250000,0.079000,38.000000,0.996700,0.620000,10.200000,6.000000,0.000000,0.000000
75%,9.100000,0.640000,0.420000,0.090000,62.000000,0.997800,0.720000,11.100000,6.000000,0.000000,1.000000
max,15.900000,1.330000,1.000000,0.611000,289.000000,1.003690,2.000000,14.900000,8.000000,1.000000,1.000000


Here, we fit the scaler before using it to transform both train and test dfs.

In [64]:
scaler = MinMaxScaler() #build scaler
scaler.fit(whole_set) #fit scaler to entire df

#transform train and test separately, both using same scaler fit from whole df
train_scaled=scaler.transform(train)
test_scaled=scaler.transform(test)

#make transformed data in a dataframe (.transform returns arrays, we want df) using old col names
train_scaled = pd.DataFrame(train_scaled, columns=train.columns)
test_scaled = pd.DataFrame(test_scaled, columns=test.columns)

From our scaled data frames, we will separate each into dataframes of attributes only and class label 'quality' only. Additionally we put the number of attributes into a variable n_inputs to be used as a parameter in the model.

In [53]:
trainScaled = train_scaled.drop('quality',axis=1) #training df without class column

testScaled = test_scaled.drop('quality',axis=1) #training df without class column

trainScaledClass = train_scaled['quality'] #training df only class column

testScaledClass = test_scaled['quality'] #testing df only class column

n_inputs = [trainScaled.shape[1]] #10 attributes = 10 input nodes

Next we build, summarize, and compile the model.

In [54]:
modelScaled = tf.keras.Sequential([tf.keras.layers.Dense(units=1,input_shape=n_inputs)]) #build model with one layer
 
modelScaled.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 1)                 11        
                                                                 
Total params: 11
Trainable params: 11
Non-trainable params: 0
_________________________________________________________________


In [55]:
modelScaled.compile(optimizer='adam',
              loss='mae',
              metrics=['accuracy'])

Finally we fit the model and view the accuracy outputs at each epoch.

In [56]:
modelScaled.fit(trainScaled, trainScaledClass, epochs=25)

Epoch 1/25
40/40 [==============================] - 0s 2ms/step - loss: 0.7182 - accuracy: 0.0063
Epoch 2/25
40/40 [==============================] - 0s 2ms/step - loss: 0.5833 - accuracy: 0.0070
Epoch 3/25
40/40 [==============================] - 0s 2ms/step - loss: 0.4642 - accuracy: 0.0094
Epoch 4/25
40/40 [==============================] - 0s 2ms/step - loss: 0.3797 - accuracy: 0.0094
Epoch 5/25
40/40 [==============================] - 0s 1ms/step - loss: 0.3274 - accuracy: 0.0109
Epoch 6/25
40/40 [==============================] - 0s 1ms/step - loss: 0.2905 - accuracy: 0.0109
Epoch 7/25
40/40 [==============================] - 0s 2ms/step - loss: 0.2607 - accuracy: 0.0102
Epoch 8/25
40/40 [==============================] - 0s 1ms/step - loss: 0.2345 - accuracy: 0.0109
Epoch 9/25
40/40 [==============================] - 0s 2ms/step - loss: 0.2117 - accuracy: 0.0109
Epoch 10/25
40/40 [==============================] - 0s 2ms/step - loss: 0.1932 - accuracy: 0.0125
Epoch 11/25
40/40 [